In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy as sp
import h5py

import holodeck as holo
import holodeck.single_sources as ss
from holodeck.constants import YR, MSOL
from holodeck import utils, detstats, plot


import hasasia.sensitivity as hsen
import hasasia.sim as hsim

# Step 1. Read in hdf file

In [ ]:
filepath = '/Users/emigardiner/GWs/holodeck/output/2023-05-16-mbp-ss19_uniform05A_n1000_r50_d20_f30_l2000_p0/'
filename = filepath+'ss_lib.hdf5'
ssfile = h5py.File(filename, 'r')
print(list(ssfile.keys()))
# print(ssfile['pta_dur'][0])
fobs = ssfile['fobs'][:]
dfobs = ssfile['dfobs'][:]
dur = ssfile['pta_dur'][0]
cad = ssfile['pta_cad'][0]
hc_ss_all = ssfile['hc_ss'][...]
hc_bg_all = ssfile['hc_bg'][...]
shape = hc_ss_all.shape
nsamp, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
print(nsamp, nfreqs, nreals, nloudest)
ssfile.close()

In [ ]:
print('samples (N):', nsamp, '\nreals (R):', nreals, '\nN,F,R,L:', hc_ss_all.shape)

# Step 2. Calibrate PTA

In [ ]:
# find best
# found this in a another notebook to be sample 30
nn=30

In [ ]:
# modify these until dp_bg = .5
npsrs=60
sigma=1.24e-8

# build PTA
phis = np.random.uniform(0, 2*np.pi, size = npsrs)
thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
# sigmas = np.ones_like(phis)*sigma
psrs = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigma,
                phi=phis, theta=thetas)

In [ ]:
dp_bg = detstats.detect_bg_pta(psrs, fobs, cad, hc_bg_all[nn])
print(np.mean(dp_bg))

# Step 3. Build PTA, Build skies, Calculate DP, Calculate DF
Step 2 was skipped, we didn't calibrate the PTA. Just built one randomly.

In [ ]:
def run_psr_calculations_and_plot(hc_ss_all, hc_bg_all, npsrs, sigma, nskies = 25, thresh=0.5,
                                  detstats_path='./'):
    """ Calculate DPs for a set of randomly placed pulsars with the number and range given.

    Parameters
    ----------

    Returns
    -------
    
    """
    shape = hc_ss_all.shape
    nsamp, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]

    # build PTA
    print('building psrs')
    phis = np.random.uniform(0, 2*np.pi, size = npsrs)
    thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
    # sigmas = np.ones_like(phis)*sigma
    psrs = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigma,
                    phi=phis, theta=thetas)

     # Build ss skies
    print('building ss skies')
    theta_ss, phi_ss, Phi0_ss, iota_ss, psi_ss = _build_skies(nfreqs, nskies, nloudest)

    # Calculate DPs 
    print('calculating SS and BG detections probabilities')
    dp_bg_all = np.zeros((nsamp, nreals)) # (N,R)
    dp_ss_all = np.zeros((nsamp, nreals, nskies)) # (N,R,S)
    df_bg_all = np.zeros(nsamp)
    df_ss_all = np.zeros(nsamp)
    for nn in range(nsamp):
        print('on nn/N = %d/%d' % (nn,nsamp))
        dp_bg_all[nn,:] = detstats.detect_bg_pta(psrs, fobs, cad, hc_bg_all[nn])
        dp_ss_all[nn,:,:] = detstats.detect_ss_pta(psrs, cad, dur, fobs, dfobs,
                                                hc_ss_all[nn], hc_bg_all[nn],
                                                theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss,
                                                iota_ss=iota_ss, psi_ss=psi_ss)
        count_bg = 0
        count_ss = 0
        for rr in range(nreals):
            if (dp_bg_all[nn,rr] > thresh):
                count_bg += 1
            for ss in range(nskies):
                if (dp_ss_all[nn,rr,ss] > thresh):
                    count_ss += 1
        df_bg_all[nn] = count_bg/nreals
        df_ss_all[nn] = count_ss/(nreals*nskies)

        fig = plot_sample_nn(fobs, hc_ss_all[nn], hc_bg_all[nn],
                         dp_ss_all[nn], dp_bg_all[nn], 
                         df_ss_all[nn], df_bg_all[nn], nn=nn)
        fname = (detstats_path+'p%06d_detprob.png' % nn) # need to make this directory
        print(fname)
        fig.savefig(fname, dpi=100)
        plt.close(fig)

    return dp_ss_all, dp_bg_all, df_ss_all, df_bg_all



def _build_skies(nfreqs, nskies, nloudest):
    theta_ss = np.random.uniform(0, np.pi, size = nfreqs * nskies * nloudest).reshape(nfreqs, nskies, nloudest)
    phi_ss = np.random.uniform(0, 2*np.pi, size = theta_ss.size).reshape(theta_ss.shape)
    Phi0_ss = np.random.uniform(0,2*np.pi, size = theta_ss.size).reshape(theta_ss.shape)
    iota_ss = np.random.uniform(0,  np.pi, size = theta_ss.size).reshape(theta_ss.shape)
    psi_ss = np.random.uniform(0,   np.pi, size = theta_ss.size).reshape(theta_ss.shape)
    return theta_ss, phi_ss, Phi0_ss, iota_ss, psi_ss
        



In [ ]:
import os
detstats_path = filepath+'/detstats/dp3_psrs40_sigma1e-6/'
os.makedirs(detstats_path)

In [ ]:
dp_ss3, dp_bg3, df_ss3, df_bg3 = run_psr_calculations_and_plot(hc_ss_all, hc_bg_all, npsrs = 40, sigma = 1e-6, 
                         nskies = 25, detstatspath=detstats_path) # this is a pretty optimistic pta

In [ ]:
fig1 = plot_detprob(dp_ss3, dp_bg3)
fig2 = plot_detfrac(df_ss3, df_bg3, thresh=0.5)
fig1.savefig(detstats_path+'allsamp_detprobs.png', dpi=300)
fig2.savefig(detstats_path+'allsamp_detfracs.png', dpi=300)
np.savez(detstats_path+'detstats.npz', dp_ss=dp_ss3, dp_bg=dp_bg3, df_ss=df_ss3, df_bg=df_bg3)

In [ ]:
'''
def _detfrac_from_reals(dp_ss_all, dp_bg_all, thresh=0.5):
    """ 
    Calculate how many realizations have a detections, to get 
    the detection probability for a sample.

    Parameters
    ----------
    dp_ss_all : (N, R, S) NDarray
    dp_bg_all : (N, R) NDarray
    thresh : scalar
        threshold DP to count as a detection

    Returns
    -------
    ss_detfrac : (N,) NDarray
        Fraction of realizations with an ss DP above thresh.
    bg_detfrac : (N,) NDarray
        Fraction of realizations with a bg DP above thresh.

    NOTE: With minimal effort this can be done without for loops, 
    just needs some count() and condition array things
    """
    nsamp = len(dp_ss_all)
    nreals = len(dp_ss_all[0])
    nskies = len(dp_ss_all[0,0])
    bg_detfrac = np.zeros(nsamp)
    ss_detfrac = np.zeros(nsamp)
    for nn in range(nsamp):
        count_bg = 0
        count_ss = 0
        for rr in range(nreals):
            if (dp_bg_all[nn,rr] > thresh):
                count_bg += 1
            for ss in range(nskies):
                if (dp_ss_all[nn,rr,ss] > thresh):
                    count_ss += 1
        bg_detfrac[nn] = count_bg/nreals
        ss_detfrac[nn] = count_ss/(nreals*nskies)

    return ss_detfrac, bg_detfrac
'''